# SA-CCR

When using *SA-CCR* the exposure at default (EAD) has to be calculated as: 

\begin{align*}
EAD &= \alpha * (RC + PFE)\\
\\
\text{where} \qquad \alpha&=1.4 \\
RC&: \text{Replacement Cost} \\
PFE&: \text{Potential Future Exposure}
\end{align*}

Let's set this relationship up in Python

In [1]:
# Imports
from datetime import datetime, timedelta
from enum import Enum
from math import exp
from scipy.stats import norm

def calculate_sa_ccr_ead(rc:float , pfe: float) -> float:
    """
    Calculate EAD of SA-CCR as defined in paragraph 186.
    
    :param rc: Replacement Cost
    :param pfe: Potential Future Exposure
    :return: Exposure at default according to SA-CCR
    """

    alpha = 1.4 #Carried over from alpha used for IMM
    return alpha*(rc+pfe)

calculate_sa_ccr_ead(10.0,15.0)

35.0

SA-CCR differentiates between the following asset classes:

* Interest rate derivatives
* Foreign exchange derivatives
* Credit derivatives
* Equity derivatives
* Commodity derivatives

Each of these asset classes is differentiated again into different hedging sets - Netting of expsoure is only possible within a common hedging set. The definition of a hedging set depends on the asset class. For FX derivatives derivatives based on the same currency pair form a hedging set. For Equity derivatives on the other hand all trades are in a common hedging set. For all asset classes, derivatives that have a basis as an underlying and volatility derivatives form two seperate hedging sets.

In [2]:
class AssetClass(Enum):
    """
    Enum for the five asset classes defined in SA-CCR. 
    """
    IR = 'Interest Rate'
    EQ = 'Equity'
    CR = 'Credit'
    CO = 'Commodity'
    FX = 'Foreign Exchange'
    

### Relation of RC and PFE
The purpose of the RC is to assess the imidiate loss suffered by the default of a counterparty. It is based on the current MtM of the derivative less the accessible collateral. If a bank has posted collateral to non-segregated accounts of a counterparty this collateral is also assumed to be lost in case of a default which increases the replacement cost.

The potential future exposure (PFE) on the other hand assesses how the RC might develop in the future. The future being defined as during the next year. If the RC today is 0 but is likely to be larger than 0 in the near future the estimated EAD should take this expected increase in RC into account.

See also Paragraph 130 and 131 of BCBS(2014):

Paragraph 130 - case without margining:

> For unmargined transactions, the *RC* intends to caputre the loss that would occur if a counterparty were to default and were closed out of its transactions immediately. The *PFE* add-on represents a potential conversative increase in expousre over a one-year time horizon from the present date (i.e. the calculation date).

Paragraph 131 - case with margining:

> For margined trades, the *RC* intends to capture the loss that would occur if a counterparty were to default at the present or at a future time, assuming that the closeout and replacement of transactions occur instantaneously. However, there may be a period (the margin period of risk) between the last exchange of collateral before default and replacement of the trades in the market. The *PFE* add-on represents the potential change in value of the trades during this time period.

## Definition of Replacement Cost (RC)

The replacement cost for unmargined trades is defined as 

\begin{align}
RC &= \max\{V-C;0\} \\
\\
\text{where} \qquad V&: \text{Value of derivative transactions} \\
C&: \text{Net collateral held according to NICA methodology}
\end{align}

### Net independendent margin amount - NICA

Independent margin is margin that exceeds the current *MtM* (i.e. the Variation Margin) of the portfolio - this could be a voluntary or accidental overcollateralization or a mantated initial margin amount. If margin is posted to segregated accounts[<sup>1</sup>](#fn1) NICA can only become positive.

<span id="fn1"><sup>1</sup> Margin on a segregated account can surely be recovered if the counterparty to which the margin has been pledged defaults. Therefore this margin can not be lost through a default of the counterparty and can only be used by the counterparty to which it has been plegded in case of a default of the pledgor. Generally speaking VM is transferred to non-segregated accounts while IM is transferred to segregated accounts.</span>

### RC under margining
To consider margining RC is adjusted to be

\begin{align}
RC &= \max\{V-C; TH+MTA-NICA;0\} \\
\\
\text{where} \qquad TH&: \text{Threshold} \\
MTA&: \text{Minimum Transfer Amount} \\
NICA&: \text{Net Independent Collateral Amount}
\end{align}

Here, the term $TH+MTA-NICA$ represents the maximum exposure that does not trigger a new variation margin call.

Finally, let's set RC up in Python:

In [3]:
def calculate_rc(v: float, c: float, th:float , mta:float, nica:float) -> float:
    """
    
    :param v: Current value of the derivative transactions in the netting set
    :param c: Haircut value of the net collateral held
    :param th: Threshold set in the collateral agreement
    :param mta: Minimum transfer amount set in the collateral agreement
    :param nica: Current net independent collateral amount (compare paragraph 143)
    :return: Replacement Cost as defined in paragraph 144
    """
    result = max(v-c, th+mta-nica, 0)
    return result

calculate_rc(2.0,3.0,4.0,5.0,6.0)

3.0

## Definition of Potential Future Exposure (PFE)

\begin{align}
PFE &= \text{multiplier} * AddOn^{\text{aggregate}} \\
\\
\text{where} \qquad AddOn^{\text{aggregate}} &: \text{aggregate add-on component} \\
\text{multiplier} &: f(V,C,AddOn^{\text{aggregate}})
\end{align}

$AddOn$ is calculated differently for each asset $a$ class. Since no netting is allowed between asset classes the aggregate is calculated as:

$$AddOn^{\text{aggregate}} = \sum_{a}AddOn^{a}$$

Collateralization is taken into account of the PFE calculation through the multiplier that uses the collateral held as an input. As overcollateralization e.g. through IM increases, the multiplier decreases. However, the multiplier is floored at 5%.

\begin{align}
\text{multiplier} &= \min \left\{ 1; Floor + (1-Floor) \exp\left(\frac{V-C}{2(1-Floor)AddOn^{\text{aggregate}}}\right) \right\} \\
\text{where} \qquad Floor &= 5\%
\end{align}

In [4]:
def multiplier(v:float,c:float, addOn_aggregate:float, floor:float = 0.05) -> float:
    """
    Multiplier calculation for SA-CCR
    
    :param v: Current value of the derivative transactions in the netting set
    :param c: Haircut value of the net collateral held
    :param addOn_aggregate: Aggregated (summed up) AddOn over all asset classes
    :param floor: Regulatory floor for the multiplier. Set to 5% in paragraph 149
    :return: Multiplier for PFE calculation according to SA-CCR
    """
    
    return min(1, floor+(1-floor)*exp((v-c)/2*(1-floor)*addOn_aggregate))

# AddOn calculation

Most of the SA-CCR logic is hidden inside the AddOn calculation. At first it is important to define the following four data parameters:

#### $M_i$ 
> Maturity of the derivative contract. If the underlying of a derivative is another derivative - e.g. in the case of a swaption the maturity date of the underlying needs to be chosen.

#### $S_i$

> For interest rate and credit derivatives the start date of the time periodreferenced by an interst rate or credit contract. If the derivtives underlying is another interest rate or credit intsrument (eg swaption or bond option) $S_i$ is the start date of the underlying instead.

#### $E_i$

> Defined as $S_i$ but referencing the end date instead of the start date.

#### $T_i$

> For options across all asset classes this is the latest contractual exercise date.

Let's include a basic Python class for a trade at this point.

In [5]:
class TradeType(Enum):
    PUT = 'Put'
    CALL = 'Call'
    LINEAR = 'Linear'
    
class TradeDirection(Enum):
    SHORT = 'Short' # A short trade looses value as the underlying rises in value
    LONG = 'Long' # A long trade rises in value as the underlying rises in value

class Trade:
    
    def __init__(self, 
                 assetClass: AssetClass,
                 tradeType: TradeType = TradeType.LINEAR,
                 tradeDirection: TradeDirection = TradeDirection.LONG,
                 m: float = None, 
                 s: float = None, 
                 e: float = None,
                 t: float = None,
                 notional: float = None):
        
        """
        Trade object for SA-CCR calculation. 
        
        :param assetClass: Asset Class. Refer to Asset Class Enum
        :param tradeType: Trade Type. Can be Put, Call or Linear
        :param tradeDirection: Trade Direction. Can be long or short
        :param m: residual maturity in years
        :param s: time until start date of underlying in years (0 if it has already started)
        :param e: time until end date of underlying in years
        :param t: time until final latest contractual 
        :param notional: notional of the trade. Denoted in domestic currency for FX trades share or commodity count for EQ and CO asset classes
        """
        
        self.assetClass = assetClass
        self.tradeType = tradeType
        self.tradeDirection = tradeDirection
        self.m = m
        self.s = s
        self.e = e
        self.t = t
        self.notional = notional
        
    def __str__(self):
        dict = {'Asset Class': self.assetClass.value,
                'Trade Type' : self.tradeType.value,
                'Trade Direction' : self.tradeDirection.value,
                'Start Date' : str(self.s),
                'End Date': str(self.e),
                'Notional': str(self.notional)}
        return str(dict)

t = Trade(assetClass = AssetClass.IR, s=0, e=2)
print(t)
del t

{'Asset Class': 'Interest Rate', 'Trade Type': 'Linear', 'Trade Direction': 'Long', 'Start Date': '0', 'End Date': '2', 'Notional': 'None'}


## Trade level adjusted notional

Each trade $i$ has a trade level adjusted notional $d_i^a$ assigned to it. This is calculated differently for the different asset classes.

#### Interest rate and credit derivatives

The notional of the trade is usually a well defined value in domestic currency for interest rate and credit derivatives. It is multiplied by a supervisory duration factor. The basic idea is, that the value of the derivative can change more the longer the remaining 

\begin{align}
d_i &= \text{Notional}_i * SD_i \\
\\
\text{where} \qquad SD_i &=\frac{\exp\left(-0.05 * S_i\right)-\exp\left(-0.05 * E_i\right)}{0.05}
\end{align}

#### FX derivatives

The notional is the current value of the foreign leg in domestic currency. Therefore it fluctuates over time with the exchange rate.

#### Equity and commodity derivatives

The notional is defined as the price of the underlying. Therfore, it fluctuates over time.

#### Notional of exotic derivatives

For more exotic derivatives which do have adjustable notionals, resetting notionals etc. detailed handling of the notional is defined in paragraph 158.

In [6]:
def trade_level_adjusted_notional(trade: Trade, underlying_price: float = None):
    """
    Calculates the trade level adjusted notional as defined in paragraph 157
    
    :param trade: Trade object for which the adjusted notional should be calculated
    :param underlying_price: 
        required for FX trades (exchange rate to domestic currency)
        required for EQ trades (current price of one share)
        required for CO trades (current price of one unit of commodity)
        not required for IR trades and CR trades
    
    :return: adjusted notional as defined in paragraph 157
    """
    
    if trade.assetClass in (AssetClass.IR, AssetClass.CR):
        sd = (exp(-0.05*trade.s)-exp(-0.05*trade.e))/0.05
        return trade.notional*sd
    
    # This assumes that trade.Notional is defined as quantity of underlying shares for equity and foreign currency notional for FX
    if trade.assetClass in (AssetClass.EQ, AssetClass.FX):
        return trade.notional*underlying_price

s = 2.0
e = 4.0
ir_t = Trade(AssetClass.IR, s=s, e=e, notional=100000)
print(ir_t)
print(trade_level_adjusted_notional(ir_t))

eq_t = Trade(AssetClass.EQ, s=s, e=e, notional=20)
print(eq_t)
print(trade_level_adjusted_notional(eq_t, underlying_price=30))

{'Asset Class': 'Interest Rate', 'Trade Type': 'Linear', 'Trade Direction': 'Long', 'Start Date': '2.0', 'End Date': '4.0', 'Notional': '100000'}
172213.3299159554
{'Asset Class': 'Equity', 'Trade Type': 'Linear', 'Trade Direction': 'Long', 'Start Date': '2.0', 'End Date': '4.0', 'Notional': '20'}
600


## Supervisory delta adjustments: $\delta_i$

For linear derivatives $\delta$ is 1 for long derivatives and -1 for short derivatives.

For options $\delta$ is defined as under Black-Scholes:

\begin{align}
\delta_{\text{long Call}} &= +\Phi\left(\frac{\ln\left(P_i / K_i \right) + 0.5 * \sigma_i^2 * T_i}{\sigma_i * \sqrt{T_i}}\right) \\
\\
\text{where} \qquad \Phi &: \text{standard normal cdf} \\
\sigma_i &: \text{supervisory volatility as defined in Table 2 in paragraph 183}
\end{align}

This delta is multiplied by -1 in case of a long Put option or a short Call option.

No detail is given at this point on the delta calculation of CDO tranches as these are not in the scope of this thesis.

In [7]:
def get_supervisory_volatility(trade: Trade) -> float:
    pass

def calculate_sa_ccr_delta(trade) -> float:
    if (trade.tradeType == TradeType.LINEAR) and (trade.tradeDirection == TradeDirection.LONG):
        return 1
    elif (trade.tradeType == TradeType.LINEAR) and (trade.tradeDirection == TradeDirection.SHORT):
        return -1
    elif trade.tradeType in (TradeType.CALL, TradeType.PUT):
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-7-48813ec166aa>, line 12)

## Risk horizon

For unmargined transaction the margining factor is


$$MF^{\text{unmargined}}_i = \sqrt{\frac{\min\left(M_i;1\text{ year}\right)}{1\text{ year}}}$$

This factor can be used to scale down a risk weight calibrated for a 1 year horizon to a shorter period.

With margining the margin period of risk (MPOR) is:

* 10 business days for small, uncleared OTC portfolios
* 5 business days for cleared derivatives
* 20 business days for netting sets with more than 5000 transactions that are not with a central counterparty
* and doubling this period for portfolios with outstanding disputes

The margining factor is then

$$ MF^{\text{margined}}_i = \frac{3}{2}\sqrt{\frac{MPOR_i}{1\text{ year}}} $$

## AddOn for interest rate derivatives

#### Step 1 - calculation of effective notional $D_{jk}^{IR}$

\begin{align}
D_{jk}^{IR} &= \sum_{i\in\left\{Ccy_j, MB_k\right\}}{\delta_i*d_i^{IR}*MF_i}
\end{align}

Here, the notation $i\in\left\{Ccy_j, MB_k\right\}$ refers to trades whose underlying is the interest rate of a common currency $j$ and which mature in a common maturity bucket $k$

In [ ]:
def interest_rate_addOn(trades):
    interest_rate_addOns = {}
    bucketed_trades = {}
    
    for t in trades:
        key = (t.currency, t.get_maturity_bucket())
        bucketed_trades[key].add(t)
    
    for key, trades in bucketed_trades.items():
        pass